#### Vit Train on Features extracted
* Create a new Dataset with the extracted features
* Create a new model
* Train the model

In [2]:
from drecg.data.utils import FeaturesDataset
train_dataset = FeaturesDataset('features_ext_vit/train_features.pt')
train_dataset_augmented = FeaturesDataset('features_ext_vit/train_features_augmented.pt')
validation_dataset = FeaturesDataset('features_ext_vit/validation_features.pt')
test_dataset = FeaturesDataset('features_ext_vit/test_features.pt')


In [57]:
import torch
import numpy as np
x_train = (train_dataset_augmented.features_a - train_dataset_augmented.features_b).numpy()
y_train = train_dataset_augmented.labels.numpy()

x_val = (validation_dataset.features_a - validation_dataset.features_b).numpy()
y_val = validation_dataset.labels.numpy()

x_test = (test_dataset.features_a - test_dataset.features_b).numpy()
y_test = test_dataset.labels.numpy()

In [59]:
x_test.shape, x_train.shape, x_val.shape

((346, 1024), (11658, 1024), (164, 1024))

In [60]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_val, label=y_val)
dtest = xgb.DMatrix(x_test, label=y_test)

In [72]:
param = { 'objective': 'binary:logistic', 
'max_depth': 4, 'eta': 0.15, 'gamma': 1.5, 'min_child_weight': 1.5, 'subsample': 0.8,}

param['eval_metric'] = ['auc', 'error']
evallist = [(dtrain, 'train'), (dval, 'eval')]
num_round = 48
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-auc:0.72341	train-error:0.29302	eval-auc:0.75506	eval-error:0.26829
[1]	train-auc:0.79764	train-error:0.24387	eval-auc:0.81536	eval-error:0.21341


/home/daniel/mambaforge/envs/recg/lib/python3.9/site-packages/xgboost/core.py:568: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[2]	train-auc:0.82359	train-error:0.23520	eval-auc:0.84600	eval-error:0.22561
[3]	train-auc:0.84873	train-error:0.21856	eval-auc:0.85128	eval-error:0.21951
[4]	train-auc:0.87108	train-error:0.18914	eval-auc:0.85901	eval-error:0.19512
[5]	train-auc:0.88169	train-error:0.17773	eval-auc:0.87098	eval-error:0.17073
[6]	train-auc:0.89183	train-error:0.17096	eval-auc:0.87545	eval-error:0.15244
[7]	train-auc:0.89940	train-error:0.16281	eval-auc:0.88028	eval-error:0.16463
[8]	train-auc:0.90490	train-error:0.15612	eval-auc:0.88288	eval-error:0.15244
[9]	train-auc:0.91052	train-error:0.15097	eval-auc:0.88355	eval-error:0.13415
[10]	train-auc:0.91449	train-error:0.14599	eval-auc:0.88355	eval-error:0.15244
[11]	train-auc:0.91692	train-error:0.14162	eval-auc:0.88712	eval-error:0.14634
[12]	train-auc:0.91980	train-error:0.13604	eval-auc:0.88980	eval-error:0.12805
[13]	train-auc:0.92256	train-error:0.13373	eval-auc:0.88757	eval-error:0.13415
[14]	train-auc:0.92502	train-error:0.13013	eval-auc:0.88601	

In [73]:
bst.eval(dtest)

'[0]\teval-error:0.20520231213872833\teval-error:0.20520231213872833'

In [4]:
import mlflow
from pathlib import Path
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = mlflow.create_experiment(
    "Feature Extraction VIT + xGBoost",
    tags={"version": "v1", "type": "vit_features_xgboost"},
)

experiment = mlflow.set_experiment("Feature Extraction VIT + xGBoost")